# Importing required Libraries

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

## Importing Dataset

In [ ]:
train_df = pd.read_csv("../input/train.csv")
train_df, val_df = train_test_split(train_df,test_size = 0.07)

Let's see the length of the question

In [ ]:
train_df.question_text.str.split().str.len().describe()

From above we can see 75% of question are less than 15 words so let's truncate  the sequence of words

In [ ]:
SEQ_LEN = 100 # we set max length of each to be 100 words

## Using glove embeddings

In [ ]:
embeddings_index = {}
f = open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')

for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:],dtype = 'float32')
    embeddings_index[word] = coefs
f.close()

Let's see a word vector for the word speech and it's length

In [ ]:
print("Lenth of vector is ",len(embeddings_index['speech']),"\n","Vector for word speech","\n",embeddings_index['speech'])

Let's see the number of word vectors found in glove embeddings

In [ ]:
len(embeddings_index)

## Data Preprocessing

Tokenizing the sentence first

In [ ]:
import re
_WORD_SPLIT = re.compile("([.,!?\"':;)(])")
_DIGIT_RE = re.compile(br"\d")
STOP_WORDS = "\" \' [ ] . , ! : ; ?".split(" ")
def basic_tokenizer(sentence):
    """Very basic tokenizer: split the sentence into a list of tokens."""
    words = []
    for space_separated_fragment in sentence.strip().split():
        words.extend(_WORD_SPLIT.split(space_separated_fragment))
        # return [w.lower() for w in words if w not in stop_words and w != '' and w != ' ']
    return [w.lower() for w in words if w != '' and w != ' ']

Converting the tokenized sentence to embeddings

In [ ]:
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = basic_tokenizer(text[:-1])[:SEQ_LEN]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (SEQ_LEN - len(embeds))
    return np.array(embeds)

Applying the preprocessing functions to train and test data

In [ ]:
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["question_text"][:3000])])
val_y = np.array(val_df["target"][:3000])

## Batching the train data to a size of 256

In [ ]:
batch_size = 256

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

## Importing required keras models and layers**

In [ ]:
from keras.models import Sequential,Model
from keras.layers import CuDNNLSTM, Dense, Bidirectional,Input,Dropout

from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers,regularizers, constraints

### Creating the Bi Directional LSTM Model

In [ ]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(128, return_sequences = True),input_shape = (SEQ_LEN,300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Running the model for 5 epochs

In [ ]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=5,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

### Creating the submission file using the test data

In [ ]:
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr

test_df = pd.read_csv("../input/test.csv")

all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

In [ ]:
y_te = (np.array(all_preds) > 0.5).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)

## Saving the model

In [ ]:
model_json = model.to_json()

In [ ]:
with open("model_questionS.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("model_questionS.h5")